## Main Model

In [6]:
import tensorflow as tf
import numpy as np
import pickle


In [7]:
def unpickle(file):
   
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [8]:
x_train=np.array([])
y_train=np.array([])
for i in range(1,6):
    
    batch = unpickle("data\cifar-10-batches-py\data_batch_%d"%(i))
    if len(x_train)==0 & len(y_train)==0:
        x_train = batch[b'data']
        y_train = batch[b'labels']
    else:
        x_train = np.concatenate((x_train, batch[b'data'])) 
        y_train = np.concatenate((y_train, batch[b'labels']))

In [9]:
y_train0 = np.empty((0,10), int)
for y in y_train:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_train0 = np.append(y_train0, np.array([indi]), axis=0)

In [11]:
test_batch = unpickle(r"data\cifar-10-batches-py\test_batch")
x_test = test_batch[b'data']
y_test = test_batch[b'labels']

In [12]:
y_test0 = np.empty((0,10), int)
for y in y_test:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_test0 = np.append(y_test0, np.array([indi]), axis=0)

## Model

In [13]:
def compute_logits(x):
    x_image = tf.reshape(x,[-1,32,32,3])
    x_image = tf.image.resize_images(x_image,[126,126])
    n1=32
    n2=64
    n3=96
    n4=128
    n5=160
    n6=192
    n7=224
    n8=256
    
    #Block1
    #cnn_1
    W_conv1 = tf.get_variable('W_conv1', shape=[2, 2, 3, n1])
    b_conv1 = tf.get_variable('b_conv1', shape=[n1])
    h_conv1 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME'), b_conv1))
    #cnn_2
    W_conv2 = tf.get_variable('W_conv2', shape=[2, 2, n1, n1])
    b_conv2 = tf.get_variable('b_conv2', shape=[n1])
    h_conv2 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME'), b_conv2))
    #fractional_max_pooling_1
    #print(type(h_conv2))
    fmp_1=tf.nn.fractional_max_pool(h_conv2,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    #tf.to_float(fmp_1)
    #print(type(fmp_1))
    

    
    #block_2
    #dropout rate 0.1 
    #cnn_3
    W_conv3 = tf.get_variable('W_conv3', shape=[2, 2, n1, n2])
    b_conv3 = tf.get_variable('b_conv3', shape=[n2])
    h_conv3 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_1, W_conv3, strides=[1, 1, 1, 1], padding='SAME'), b_conv3))
    #dropout_1
    d_1=tf.nn.dropout(h_conv3,0.9)
    #cnn_4
    W_conv4 = tf.get_variable('W_conv4', shape=[2, 2, n2, n2])
    b_conv4 = tf.get_variable('b_conv4', shape=[n2])
    h_conv4 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_1, W_conv4, strides=[1, 1, 1, 1], padding='SAME'), b_conv4))
    #dropout_2
    d_2=tf.nn.dropout(h_conv4,0.9)
    #fractional_max_pooling_
    fmp_2 = tf.nn.fractional_max_pool(d_2,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    
    
    #block_3
    #dropout=0.2
    #cnn_5
    W_conv5 = tf.get_variable('W_conv5', shape=[2, 2, n2, n3])
    b_conv5 = tf.get_variable('b_conv5', shape=[n3])
    h_conv5 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_2, W_conv5, strides=[1, 1, 1, 1], padding='SAME'), b_conv5))
    #dropout_3
    d_3=tf.nn.dropout(h_conv5,0.8)
    #cnn_6
    W_conv6 = tf.get_variable('W_conv6', shape=[2, 2, n3, n3])
    b_conv6 = tf.get_variable('b_conv6', shape=[n3])
    h_conv6 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_3, W_conv6, strides=[1, 1, 1, 1], padding='SAME'), b_conv6))
    #dropout_4
    d_4=tf.nn.dropout(h_conv6,0.8)
    #fractional_max_pooling_
    fmp_3 = tf.nn.fractional_max_pool(d_4,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    

    
    #block_4
    #dropout=0.3
    #cnn_7
    W_conv7 = tf.get_variable('W_conv7', shape=[2, 2, n3, n4])
    b_conv7 = tf.get_variable('b_conv7', shape=[n4])
    h_conv7 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_3, W_conv7, strides=[1, 1, 1, 1], padding='SAME'), b_conv7))
    #dropout_5
    d_5=tf.nn.dropout(h_conv7,0.7)
    #cnn_8
    W_conv8 = tf.get_variable('W_conv8', shape=[2, 2, n4, n4])
    b_conv8 = tf.get_variable('b_conv8', shape=[n4])
    h_conv8 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_5, W_conv8, strides=[1, 1, 1, 1], padding='SAME'), b_conv8))
    #dropout_6
    d_6=tf.nn.dropout(h_conv8,0.7)
    #fractional_max_pooling_
    fmp_4 = tf.nn.fractional_max_pool(d_6,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    #block_5
    #dropout=0.4
    #cnn_9
    W_conv9 = tf.get_variable('W_conv9', shape=[2, 2, n4, n5])
    b_conv9 = tf.get_variable('b_conv9', shape=[n5])
    h_conv9 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_4, W_conv9, strides=[1, 1, 1, 1], padding='SAME'), b_conv9))
    #dropout_7
    d_7=tf.nn.dropout(h_conv9,0.6)
    #cnn_10
    W_conv10 = tf.get_variable('W_conv10', shape=[2, 2, n5, n5])
    b_conv10 = tf.get_variable('b_conv10', shape=[n5])
    h_conv10 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_7, W_conv10, strides=[1, 1, 1, 1], padding='SAME'), b_conv10))
    #dropout_8
    d_8=tf.nn.dropout(h_conv10,0.6)
    #fractional_max_pooling_
    fmp_5 = tf.nn.fractional_max_pool(d_8,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
   


    #block_6
    #dropout=0.5
    #cnn_11
    W_conv11 = tf.get_variable('W_conv11', shape=[2, 2, n5, n6])
    b_conv11 = tf.get_variable('b_conv11', shape=[n6])
    h_conv11 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_5, W_conv11, strides=[1, 1, 1, 1], padding='SAME'), b_conv11))
    #dropout_9
    d_9=tf.nn.dropout(h_conv11,0.5)
    #cnn_12
    W_conv12 = tf.get_variable('W_conv12', shape=[2, 2, n6, n6])
    b_conv12 = tf.get_variable('b_conv12', shape=[n6])
    h_conv12 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_9, W_conv12, strides=[1, 1, 1, 1], padding='SAME'), b_conv12))
    #dropout_10
    d_10=tf.nn.dropout(h_conv12,0.5)
    #fractional_max_pooling_
    fmp_6 = tf.nn.fractional_max_pool(d_10,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    
    #block_7
    #dropout=0.5
    #cnn_13
    W_conv13 = tf.get_variable('W_conv13', shape=[2, 2, n6, n7])
    b_conv13 = tf.get_variable('b_conv13', shape=[n7])
    h_conv13 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_6, W_conv13, strides=[1, 1, 1, 1], padding='SAME'), b_conv13))
    #dropout_11
    d_11=tf.nn.dropout(h_conv13,0.5)
    #cnn_14
    W_conv14 = tf.get_variable('W_conv14', shape=[2, 2, n7, n7])
    b_conv14 = tf.get_variable('b_conv14', shape=[n7])
    h_conv14 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_11, W_conv14, strides=[1, 1, 1, 1], padding='SAME'), b_conv14))
    #dropout_12
    d_12=tf.nn.dropout(h_conv14,0.5)
    #fractional_max_pooling_
    fmp_7 = tf.nn.fractional_max_pool(d_12,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    
    #block_8
    #cnn_15
    W_conv15 = tf.get_variable('W_conv15', shape=[2, 2, n7, n8])
    b_conv15 = tf.get_variable('b_conv15', shape=[n8])
    h_conv15 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_7, W_conv15, strides=[1, 1, 1, 1], padding='SAME'), b_conv15))
    #dropout_13
    d_13=tf.nn.dropout(h_conv15,0.5)
    #cnn_16
    W_conv16 = tf.get_variable('W_conv16', shape=[2, 2, n8, n8])
    b_conv16 = tf.get_variable('b_conv16', shape=[n8])
    h_conv16 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_13, W_conv16, strides=[1, 1, 1, 1], padding='SAME'), b_conv16))
    #dropout_2
    d_14=tf.nn.dropout(h_conv16,0.5)
    #fractional_max_pooling_
    fmp_8 = tf.nn.fractional_max_pool(d_14,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]    
    
    
    #last_cnn
    W_conv = tf.get_variable('W_conv', shape=[2, 2, n8, n8])
    b_conv = tf.get_variable('b_conv', shape=[n8])
    h_conv = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_8, W_conv, strides=[1, 2, 2, 1], padding='SAME'), b_conv))

    # fc layer to logits (2*2 )
    h_flat = tf.reshape(h_conv, [-1, 3*3*n8])
    W_fc1 = tf.get_variable('W_fc1', shape=[3*3*n8, 10])
    b_fc1 = tf.get_variable('b_fc1', shape=[10])

    logits = tf.add(tf.matmul(h_flat, W_fc1), b_fc1, name='logits')
    
    return(logits)

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy
    
    

In [22]:
dir_name = 'logs/scratch04x/summary'
with tf.Graph().as_default():

    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')

    
 
    with tf.name_scope('model'):
        logits = compute_logits(x)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):

        opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_test = tf.summary.FileWriter(dir_name+'/test', sess.graph)
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(0,20001):
            _ , summary = sess.run((train_step, summary_op),
                                feed_dict={x: x_train, y: y_train0})
            if i%100 == 0: 
                (test_error, summary) = sess.run((accuracy,summary_op), {x:x_test, y:y_test0})
                print("\rAfter step {0:3d}, test accuracy {1:0.4f}".format(i, test_error), flush=True)
                summary_writer_test.add_summary(summary, i)
    

ResourceExhaustedError: OOM when allocating tensor with shape[50000,126,126,3]
	 [[Node: model/ResizeBilinear = ResizeBilinear[T=DT_FLOAT, align_corners=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](summaries/Reshape, model/size)]]

Caused by op 'model/ResizeBilinear', defined at:
  File "C:\Users\Sherlock\Anaconda3\envs\AdvML\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Sherlock\Anaconda3\envs\AdvML\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sherlock\Anaconda3\envs\AdvML\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Sherlock\Anaconda3\envs\AdvML\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Sherlock\Anaconda3\envs\AdvML\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Sherlock\Anaconda3\en